## 실행 가능성 확인하기

In [1]:
import glob
import os
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

### 파라미터

In [18]:
EPOCHS = 10
DATASET_PATH = 'dataset/2'
DATASET_OK_PATTERN = DATASET_PATH + '/OK/*.png'
DATASET_FAIL_PATTERN = DATASET_PATH + '/FAIL/*.png'
RESULT_SAVE_PATH = DATASET_PATH + '/results'

In [19]:
DATASET_OK_PATTERN, DATASET_FAIL_PATTERN, RESULT_SAVE_PATH

('dataset/2/OK/*.png', 'dataset/2/FAIL/*.png', 'dataset/2/results')

## 단순 모델 설정

In [6]:
def Model():
    return Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
        MaxPool2D(),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPool2D(),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPool2D(),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])

### 데이터셋 불러오기

In [7]:
def preprocess(file_name):
    img = tf.io.read_file(file_name)
    img = tf.image.decode_png(img, channels=1)
    return tf.image.convert_image_dtype(img, tf.float32)

In [9]:
ok_list = glob.glob(DATASET_OK_PATTERN)
ds_ok = tf.data.Dataset.list_files(ok_list)
ds_ok_label = tf.data.Dataset.from_tensor_slices([0] * len(ok_list))

ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label))

In [10]:
fail_list = glob.glob(DATASET_FAIL_PATTERN)
ds_fail = tf.data.Dataset.list_files(fail_list)
ds_fail_label = tf.data.Dataset.from_tensor_slices([1] * len(fail_list))

ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))

In [11]:
ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

### Train, Valid 데이터셋 나누기

In [12]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32)
ds_valid = ds.skip(train_size).batch(32)

### 모델 생성 및 학습

In [15]:
model = Model()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
h = model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS)

Epoch 1/10
21/21 [==============================] - 65s 2s/step - loss: 0.4918 - accuracy: 0.8742 - val_loss: 0.3940 - val_accuracy: 0.8662
Epoch 2/10
21/21 [==============================] - 9s 248ms/step - loss: 0.3748 - accuracy: 0.8831 - val_loss: 0.3891 - val_accuracy: 0.8662
Epoch 3/10
21/21 [==============================] - 9s 253ms/step - loss: 0.3452 - accuracy: 0.8856 - val_loss: 0.4153 - val_accuracy: 0.8486
Epoch 4/10
21/21 [==============================] - 9s 245ms/step - loss: 0.3458 - accuracy: 0.8799 - val_loss: 0.3090 - val_accuracy: 0.8944
Epoch 5/10
21/21 [==============================] - 9s 250ms/step - loss: 0.3757 - accuracy: 0.8722 - val_loss: 0.4790 - val_accuracy: 0.8451
Epoch 6/10
21/21 [==============================] - 9s 250ms/step - loss: 0.3924 - accuracy: 0.8663 - val_loss: 0.3191 - val_accuracy: 0.9085
Epoch 7/10
21/21 [==============================] - 9s 246ms/step - loss: 0.3417 - accuracy: 0.8756 - val_loss: 0.3402 - val_accuracy: 0.8697
Epoch 8/

## 결과 저장

In [17]:
def mkdir(path):
    if os.path.exists(path) is False:
        os.mkdir(path)

In [22]:
mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH + '/TP')
mkdir(RESULT_SAVE_PATH + '/TN')
mkdir(RESULT_SAVE_PATH + '/FP')
mkdir(RESULT_SAVE_PATH + '/FN')

In [23]:
index = 0
for imgs, labels in ds_valid:
    preds = model.predict(imgs)
    for idx in range(imgs.shape[0]):
        gt = labels[idx].numpy()
        y = preds[idx]
        if gt == 1 and y > 0.5:
            path = RESULT_SAVE_PATH + '/TP'
        elif gt == 1 and y <= 0.5:
            path = RESULT_SAVE_PATH + '/FN'
        elif gt == 0 and y > 0.5:
            path = RESULT_SAVE_PATH + '/FP'
        else:
            path = RESULT_SAVE_PATH + '/TN'
        cv2.imwrite(path + '/%.4f_%04d.png' % (y, index), imgs[idx].numpy() * 255)
        index += 1